In [9]:
import torch
import json
import gensim.downloader as api
import numpy as np
import pickle
# from google.colab import drive
import tensorflow as tf
# from tensorflow.keras.utils import tf_utils
from tensorflow import keras
from keras.utils import to_categorical
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed
from keras.optimizers import Adam
import keras.backend as K
import keras.layers as L
# import tensorflow_addons as tfa

from tensorflow_addons.text import crf_log_likelihood, crf_decode
from sklearn.metrics import f1_score

# drive.mount('/content/drive')

In [10]:
with open('Json Task1/NER_train.json', 'r') as f:
    task1_train_data = json.load(f)

with open('Json Task1/NER_val.json', 'r') as f:
    task1_val_data = json.load(f)

with open('Json Task1/NER_test.json', 'r') as f:
    task1_test_data = json.load(f)

with open('Json Task2/ATE_train.json', 'r') as f:
    task2_train_data = json.load(f)

with open('Json Task2/ATE_val.json', 'r') as f:
    task2_val_data = json.load(f)

with open('Json Task2/ATE_test.json', 'r') as f:
    task2_test_data = json.load(f)

with open('glove_embedding.pkl', 'rb') as pickle_file:
    glove_embeddings = pickle.load(pickle_file)
    
with open('fast_text_embedding.pkl', 'rb') as pickle_file:
    fast_text_embedding = pickle.load(pickle_file)

with open('word2vec_embeddings.pkl', 'rb') as pickle_file:
    word2vec_embeddings = pickle.load(pickle_file)

bio_mapping_task1 = {'B_ORG': 0, 'I_ORG': 1, 'B_RESPONDENT': 2, 'I_RESPONDENT': 3, 'B_JUDGE': 4, 'I_JUDGE': 5,
               'B_STATUTE': 6, 'I_STATUTE': 7, 'B_OTHER_PERSON': 8, 'I_OTHER_PERSON': 9, 'B_COURT': 10, 'I_COURT': 11,
               'B_GPE': 12, 'I_GPE': 13, 'B_PETITIONER': 14, 'I_PETITIONER': 15, 'B_WITNESS': 16, 'I_WITNESS': 17,
               'B_CASE_NUMBER': 18, 'I_CASE_NUMBER': 19, 'B_PRECEDENT': 20, 'I_PRECEDENT': 21, 'B_DATE': 22, 'I_DATE': 23,
               'B_PROVISION': 24, 'I_PROVISION': 25, 'O': 26}
bio_mapping_task2 = {'O' : 0, 'I' : 1, 'B' : 2}

In [17]:
# word2vec_model = api.load('word2vec-google-news-300')

In [14]:
# word2vec = torch.FloatTensor(word2vec_model.vectors)
# word2vec_u_ = np.concatenate((word2vec, np.zeros((1, 300), dtype = 'float32')), axis = 0)

# word_to_index_w2v = {word: index for index, word in enumerate(word2vec_model.index_to_key)}
# word_to_index_w2v['<unk>'] = len(word_to_index_w2v)
# vocab_size_w2v = len(word_to_index_w2v)

# word2vec_embeddings = {word: word2vec_u_[index] for word, index in word_to_index_w2v.items()}

NameError: name 'word2vec_model' is not defined

In [11]:
word_to_index_w2v = {}
ind = 0
word2vec_embeddings['<unk>'] = np.zeros((1, 300), dtype = 'float32')

for word in word2vec_embeddings.keys():
  word_to_index_w2v[word] = ind
  ind = ind + 1

vocab_size_w2v = len(word_to_index_w2v)

In [12]:
word_to_index_glove = {}
ind = 0
glove_embeddings['<unk>'] = np.zeros((1, 300), dtype = 'float32')

for word in glove_embeddings.keys():
  word_to_index_glove[word] = ind
  ind = ind + 1

vocab_size_glove = len(word_to_index_glove)

In [13]:
word_to_index_fasttext = {}
ind = 0
fast_text_embedding['<unk>'] = np.zeros((1, 300), dtype = 'float32')

for word in fast_text_embedding.keys():
  word_to_index_fasttext[word] = ind
  ind = ind + 1

vocab_size_fasttext = len(word_to_index_fasttext)

In [14]:
def process_data(dataset,task,split, embedding_type):
    sentences = []
    labels = []
    max_length_train_1 = 80
    max_length_train_2 = 83
    # max_length_test_1 = 52
    # max_length_test_2 = 71
    max_seq_len = 0

    if task == 1:
        bio_mapping = bio_mapping_task1
        max_seq_len = max_length_train_1
    else:
        bio_mapping = bio_mapping_task2
        max_seq_len = max_length_train_2

# choosing embedding type
    if embedding_type == "word2vec":
        word_to_index = word_to_index_w2v

    if embedding_type == "glove":
        word_to_index = word_to_index_glove

    else:
        word_to_index = word_to_index_fasttext

    for index in dataset.keys():

        dic = dataset[index]
        tags = [bio_mapping[label] for label in dic['labels']]
        sentence = dic['text'].split(' ')
        sequence = []
        for word in sentence:
            if word in word_to_index:
                sequence.append(word_to_index[word])
            else:
                sequence.append(word_to_index['<unk>'])
        if(len(sequence) >= max_seq_len):
            continue

        if(len(sequence) < max_seq_len):
            for i in range(len(sequence), max_seq_len):
                sequence.append(word_to_index['<unk>'])
                tags.append(bio_mapping['O'])
                # tags.append(3)

        sentences.append(sequence)
        labels.append(tags)

    tag_size = len(bio_mapping)
    labels = [to_categorical(i, num_classes = tag_size) for i in labels]

    return np.array(sentences), np.asarray(labels)

In [15]:
X_train, y_train = process_data(task1_train_data, 1, split = "train", embedding_type = "glove")
X_val, y_val = process_data(task1_val_data, 1, split = "val", embedding_type = "glove")
X_test, y_test = process_data(task1_test_data, 1, split = "test", embedding_type = "glove")


In [16]:
class CRF(L.Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):

        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim)
        self.sparse_target = sparse_target
        self.input_spec = L.InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = L.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(base_config, **config)

In [17]:
def create_model(vocab_size, max_length, embedding_dim, word_index, tag_index, emb_type):

    if emb_type == "word2vec":
        embeddings_index = word2vec_embeddings

    elif emb_type == "glove":
        embeddings_index = glove_embeddings
    else:
        embeddings_index = fast_text_embedding

    embeddings_index = {}
    embeddings_matrix = np.zeros((vocab_size, embedding_dim))

    for word, i in word_index.items():
        if i > vocab_size:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector

    model = Sequential()

    model.add(Embedding(input_dim = vocab_size, output_dim = embedding_dim,
                    input_length = max_length, weights = [embeddings_matrix], mask_zero = True))

    model.add(Bidirectional(LSTM(units = 100, return_sequences = True,
                             recurrent_dropout = 0.01)))
    model.add(Bidirectional(LSTM(units = 30, return_sequences = True,
                             recurrent_dropout = 0.1)))
    model.add(TimeDistributed(Dense(len(tag_index))))

    crf = CRF(len(tag_index), sparse_target=True)
    model.add(crf)
    # model.compile(optimizer = 'adam', loss = crf.loss, metrics = [crf.accuracy])
    # model.summary()

    return model

In [18]:

def calculate_f1_score(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_argmax = np.argmax(y_pred, axis=-1)
    y_true_argmax = np.argmax(y_test, axis=-1)
    # print(X_test.shape, y_pred.shape, y_test.shape, y_pred_argmax.shape, y_true_argmax.shape)
    f1 = f1_score(y_true_argmax.flatten(), y_pred_argmax.flatten(), average='macro')
    return f1


In [19]:
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val, X_test, y_test):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.X_test = X_test
        self.y_test = y_test
        self.train_losses = []
        self.val_losses = []
        self.train_f1_scores = []
        self.val_f1_scores = []

    def on_epoch_end(self, epoch, logs=None):
        # Training loss
        train_loss = logs['loss']
        self.train_losses.append(train_loss)

        # Training F1 score
        train_f1_score = calculate_f1_score(self.model, X_train, y_train)
        self.train_f1_scores.append(train_f1_score)

        # Validation loss
        t = self.model.evaluate(self.X_val, self.y_val, verbose=0)
        # print(t)
        val_loss = t[0]
        self.val_losses.append(val_loss)

        # Validation F1 score
        val_f1_score = calculate_f1_score(self.model, self.X_val, self.y_val)
        self.val_f1_scores.append(val_f1_score)

        # Print metrics
        print(f'Epoch {epoch + 1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Train F1 Score: {train_f1_score:.4f} - Val F1 Score: {val_f1_score:.4f}')

In [20]:
import tensorflow as tf

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using GPU.")
else:
    print("TensorFlow is using CPU.")


TensorFlow is using CPU.


In [25]:
model = create_model(vocab_size = vocab_size_fasttext, max_length = 80, embedding_dim = 300, word_index = word_to_index_glove, tag_index = bio_mapping_task1, emb_type = "glove")


model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3), loss=model.layers[-1].loss, metrics=[model.layers[-1].accuracy])

metrics_callback = MetricsCallback(X_val, y_val, X_test, y_test)

history = model.fit(X_train, y_train, batch_size = 16, epochs = 11, validation_data = (X_val, y_val), verbose = 0, callbacks = [metrics_callback])



44/44 [==============================] - 3s 59ms/step
Epoch 1 - Train Loss: 23.6749 - Val Loss: 166.2344 - Train F1 Score: 0.0982 - Val F1 Score: 0.0966


In [24]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose = 0)
test_f1_score = calculate_f1_score(model, X_test, y_test)
print(f'Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.4f} - Test F1 Score: {test_f1_score:.4f}')

29/29 [==============================] - 2s 61ms/step
Test Loss: 169.0938 - Test Accuracy: 0.0985 - Test F1 Score: 0.5242
